# Aspect-Aware Sentiment Modeling for Educational Feedback

Formalizes the research-exchange idea with baselines, transformers, prompting, robustness, and explainability.

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = Path("data_feedback.xlsx")
RANDOM_SEED = 42

## 2. Load and audit

In [ ]:
df = pd.read_excel(DATA_PATH)
df = df.rename(columns={"teacher/course": "topic"})
df.head()

In [ ]:
print("Class distribution (sentiment):")
print(df["sentiment"].value_counts())
print("
Topic distribution:")
print(df["topic"].value_counts())

## 3. Split

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.25, stratify=df["sentiment"], random_state=RANDOM_SEED)
train_df.shape, val_df.shape

## 4. Helpers

In [ ]:
def evaluate(model, x_train, y_train, x_val, y_val, label):
    model.fit(x_train, y_train)
    preds = model.predict(x_val)
    print(f"=== {label} ===")
    print(classification_report(y_val, preds))
    cm = confusion_matrix(y_val, preds, labels=sorted(y_val.unique()))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=sorted(y_val.unique()), yticklabels=sorted(y_val.unique()))
    plt.title(label)
    plt.xlabel("Pred")
    plt.ylabel("True")
    plt.show()
    return preds

def build_aspect_prompt(texts, aspects):
    return [f"[ASPECT={a}] {t}" for a, t in zip(aspects, texts)]

## 5. Word TF-IDF baseline

In [ ]:
word_clf = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
_ = evaluate(word_clf, train_df["comments"], train_df["sentiment"], val_df["comments"], val_df["sentiment"], "Word TF-IDF")

## 6. Character TF-IDF baseline

In [ ]:
char_clf = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", ngram_range=(3,5), min_df=1)), ("clf", LogisticRegression(max_iter=200, class_weight="balanced"))])
_ = evaluate(char_clf, train_df["comments"], train_df["sentiment"], val_df["comments"], val_df["sentiment"], "Char TF-IDF")

## 7. Aspect-prompted TF-IDF

In [ ]:
prompted_train = build_aspect_prompt(train_df["comments"], train_df["aspect"])
prompted_val = build_aspect_prompt(val_df["comments"], val_df["aspect"])
asp_word_clf = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
_ = evaluate(asp_word_clf, prompted_train, train_df["sentiment"], prompted_val, val_df["sentiment"], "Aspect-prompted TF-IDF")

## 8. Sentence-embedding classifier

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler

embedder = SentenceTransformer("all-mpnet-base-v2")
train_vecs = embedder.encode(train_df["comments"].tolist(), show_progress_bar=False)
val_vecs = embedder.encode(val_df["comments"].tolist(), show_progress_bar=False)
scaler = StandardScaler(with_mean=False)
train_vecs_scaled = scaler.fit_transform(train_vecs)
val_vecs_scaled = scaler.transform(val_vecs)
embed_clf = LogisticRegression(max_iter=1000, class_weight="balanced")
embed_clf.fit(train_vecs_scaled, train_df["sentiment"])
preds = embed_clf.predict(val_vecs_scaled)
print(classification_report(val_df["sentiment"], preds))

## 9. Transformer fine-tuning

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
label2id = {l:i for i, l in enumerate(sorted(df["sentiment"].unique()))}
id2label = {i:l for l, i in label2id.items()}

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_ds = Dataset.from_dict({"text": train_df["comments"].tolist(), "label": [label2id[x] for x in train_df["sentiment"]]})
val_ds = Dataset.from_dict({"text": val_df["comments"].tolist(), "label": [label2id[x] for x in val_df["sentiment"]]})
train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)
train_ds.set_format("torch")
val_ds.set_format("torch")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id), id2label=id2label, label2id=label2id)
args = TrainingArguments(output_dir="./sentiment_distilbert", learning_rate=2e-5, per_device_train_batch_size=8, per_device_eval_batch_size=16, num_train_epochs=5, evaluation_strategy="epoch", save_strategy="epoch", load_best_model_at_end=True, weight_decay=0.01, logging_steps=10)
trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds, tokenizer=tokenizer)
# trainer.train()
# trainer.evaluate()

## 10. Aspect-aware transformer (multi-task head)

In [ ]:
from transformers import AutoModel
import torch
import torch.nn as nn

class MultiTaskModel(nn.Module):
    def __init__(self, base_model_name, num_sentiment, num_aspect):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(base_model_name)
        hidden = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(self.encoder.config.dropout)
        self.sentiment_head = nn.Linear(hidden, num_sentiment)
        self.aspect_head = nn.Linear(hidden, num_aspect)
    def forward(self, input_ids=None, attention_mask=None, labels=None, aspect_labels=None):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0]
        pooled = self.dropout(pooled)
        sent_logits = self.sentiment_head(pooled)
        asp_logits = self.aspect_head(pooled)
        loss = None
        if labels is not None and aspect_labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(sent_logits, labels) + loss_fn(asp_logits, aspect_labels)
        return {"loss": loss, "logits": sent_logits, "aspect_logits": asp_logits}

asp_label2id = {l:i for i, l in enumerate(sorted(df["aspect"].unique()))}
mt_model = MultiTaskModel(model_name, num_sentiment=len(label2id), num_aspect=len(asp_label2id))
# To train, subclass Trainer to pass both labels.

## 11. Cross-aspect generalization

In [ ]:
teacher_df = df[df["topic"] == "teacher"]
course_df = df[df["topic"] == "course"]
teacher_train, teacher_val = train_test_split(teacher_df, test_size=0.3, random_state=RANDOM_SEED)
course_train, course_val = train_test_split(course_df, test_size=0.3, random_state=RANDOM_SEED)
transfer_clf = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,2))), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
transfer_clf.fit(teacher_train["comments"], teacher_train["sentiment"])
course_preds = transfer_clf.predict(course_val["comments"])
print("Teacher->Course transfer")
print(classification_report(course_val["sentiment"], course_preds))

## 12. Data augmentation (synonyms)

In [ ]:
import random
synonym_map = {"good": ["great", "nice"], "great": ["excellent"], "helpful": ["supportive"], "humor": ["wit"], "knowledge": ["expertise"]}

def augment_comment(text):
    tokens = text.split()
    augmented = []
    for tok in tokens:
        key = tok.lower().strip(".,")
        if key in synonym_map and random.random() < 0.3:
            augmented.append(random.choice(synonym_map[key]))
        else:
            augmented.append(tok)
    return " ".join(augmented)
augmented_df = train_df.copy()
augmented_df["comments"] = augmented_df["comments"].apply(augment_comment)
augmented_df["is_aug"] = True
combined = pd.concat([train_df.assign(is_aug=False), augmented_df])
combined.head()

## 13. Zero/low-shot prompting baseline

In [ ]:
from transformers import pipeline
prompt_template = "Classify the sentiment (positive/neutral/negative) of this student feedback about {aspect} and explain briefly: {text}"
zero_shot_clf = pipeline("text-classification", model="facebook/bart-large-mnli")
example = val_df.iloc[0]
text = prompt_template.format(aspect=example["aspect"], text=example["comments"])
print(zero_shot_clf(text))

## 14. Error analysis

In [ ]:
def collect_errors(model, x_val, y_val):
    preds = model.predict(x_val)
    return pd.DataFrame({"text": x_val, "true": y_val, "pred": preds}).query("true != pred")
err_df = collect_errors(word_clf, val_df["comments"], val_df["sentiment"])
err_df.head()

## 15. CLI baseline

In [ ]:
import argparse

def run_cli():
    parser = argparse.ArgumentParser(description="Aspect-aware sentiment experiments")
    parser.add_argument("--use_aspect", action="store_true", help="prepend aspect tokens")
    args = parser.parse_args(args=[])
    x_train = train_df["comments"] if not args.use_aspect else build_aspect_prompt(train_df["comments"], train_df["aspect"])
    x_val = val_df["comments"] if not args.use_aspect else build_aspect_prompt(val_df["comments"], val_df["aspect"])
    model = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,2))), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
    evaluate(model, x_train, train_df["sentiment"], x_val, val_df["sentiment"], "CLI run")

# run_cli()

---